In [2]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [3]:
import pymc3 as pm

In [4]:
np.random.seed(23) # from random.org

# 3.1. Individual GLAM estimation prediction

## Load data

In [5]:
# Load data
sufix = '_individual_Dislike_NoBin_Gamma-11_NUTS_31'
data = pd.read_csv('data/FF2018_data/GlamDataFF2018_Dislike_NoBin_31.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,0.95,2.00,0.745415,0.254585
1,0,1,1,3424,2.30,1.70,0.410720,0.589280
2,0,2,1,3691,1.70,1.25,0.330549,0.669451
3,0,3,0,8144,1.55,2.30,0.592345,0.407655
4,0,4,0,6559,2.00,2.00,0.641717,0.358283


## Split data in training and test sets

In [6]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1860 trials) and test (1860 trials) sets...


In [7]:
train_data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,0.95,2.00,0.745415,0.254585
2,0,2,1,3691,1.70,1.25,0.330549,0.669451
4,0,4,0,6559,2.00,2.00,0.641717,0.358283
6,0,6,1,2470,1.55,1.10,0.329730,0.670270
8,0,8,1,4669,1.55,1.70,0.419431,0.580569
10,0,10,0,3301,1.10,1.10,0.605079,0.394921
12,0,12,1,3117,1.85,1.10,0.506602,0.493398
14,0,14,0,2754,1.25,1.55,0.596632,0.403368
16,0,16,1,3931,1.85,1.70,0.338392,0.661608
18,0,18,0,5674,1.70,1.40,0.410655,0.589345


## Hierarchical GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_Ff2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
Generating single subject models for 31 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 31 model(s) using NUTS...
  Fitting model 1 of 31...


INFO (theano.gof.compilelock): Waiting for existing lock by process '1957' (I am process '1906')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/pradyumna/.theano/compiledir_Darwin-18.5.0-x86_64-i386-64bit-i386-3.7.2-64/lock_dir
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [12:10<00:00,  4.76draws/s]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The acceptance probability does not match the target. It is 0.9462713531338076, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The acceptance probability does not match the target. It is 0.7179011664283137, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase tar

  Fitting model 2 of 31...


...

  Fitting model 30 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [10:38<00:00,  4.64draws/s]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 31 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [01:36<00:00, 124.89draws/s]
The acceptance probability does not match the target. It is 0.9134653091990405, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.635903160690723, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


/!\ Automatically setting parameter precision...


In [8]:
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_full'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

,SNR,b,gamma,p_error,s,t0,tau,v
0,44.47,1.0,0.11,0.05,0.002738,[0.0],0.43,0.000072
1,37.91,1.0,0.46,0.05,0.004251,[0.0],0.45,0.000055
2,143.44,1.0,-0.94,0.05,0.008393,[0.0],0.01,0.000049
3,63.36,1.0,-0.12,0.05,0.009441,[0.0],0.96,0.000046
4,158.77,1.0,-0.98,0.05,0.010653,[0.0],1.21,0.000060
5,155.27,1.0,-0.99,0.05,0.008886,[0.0],1.63,0.000057
6,140.42,1.0,-0.98,0.05,0.009101,[0.0],1.52,0.000066
7,135.05,1.0,-0.99,0.05,0.006417,[0.0],4.22,0.000051
8,71.07,1.0,0.32,0.05,0.005421,[0.0],0.30,0.000077
9,108.45,1.0,-0.91,0.05,0.007398,[0.0],4.52,0.000067


In [10]:
full_params = pd.DataFrame(glam_full.estimates)
full_params.to_csv(str('results/params_estimates/params_glam_FF2019_full'+sufix+'.csv'))

In [11]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_F2019_full'+ sufix +'.npy'))

Computing WAIC scores for full model...


/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance

In [12]:
# Compute WAICs
np.save(str('results/waic/glam_FF2019_full'+ sufix +'.npy'), glam_full.waic)

In [13]:
waic_values = pd.DataFrame(glam_full.waic)
waic_values.to_csv(str('results/params_estimates/waic_estimate'+sufix+'.csv'))

In [14]:
len(glam_full.waic)

31

In [15]:
glam_full.waic

array([[3.65550753e+03, 0.00000000e+00, 1.27478507e+03, 1.00000000e+00],
       [1.79885367e+03, 0.00000000e+00, 3.52085826e+02, 1.00000000e+00],
       [1.11519302e+03, 0.00000000e+00, 2.21657604e+00, 1.00000000e+00],
       [2.54769680e+03, 0.00000000e+00, 7.15650377e+02, 1.00000000e+00],
       [1.48524821e+03, 0.00000000e+00, 1.93035229e+02, 1.00000000e+00],
       [1.08315340e+03, 0.00000000e+00, 2.19585967e+00, 1.00000000e+00],
       [1.07784386e+03, 0.00000000e+00, 2.21666189e+00, 1.00000000e+00],
       [1.03855843e+03, 0.00000000e+00, 1.80156400e+00, 1.00000000e+00],
       [1.21038965e+03, 0.00000000e+00, 7.68457581e+01, 1.00000000e+00],
       [1.04636916e+03, 0.00000000e+00, 1.89494615e+00, 1.00000000e+00],
       [2.90979033e+03, 0.00000000e+00, 9.35965946e+02, 1.00000000e+00],
       [1.05271372e+03, 0.00000000e+00, 2.52473750e+00, 1.00000000e+00],
       [4.49750279e+03, 0.00000000e+00, 1.71218814e+03, 1.00000000e+00],
       [1.13545581e+03, 0.00000000e+00, 2.27437284e

In [16]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_FF2019_full'+ sufix +'.npy'), glam_full.loo)

AttributeError: 'list' object has no attribute 'nchains'

In [17]:
glam_full.loo

AttributeError: 'GLAM' object has no attribute 'loo'

In [18]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,subject,trial,repeat,choice,rt,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,0.0,1.0,2759.0,2.3,0.41072,1.7,0.58928
1,0.0,0.0,1.0,1.0,2689.0,2.3,0.41072,1.7,0.58928
2,0.0,0.0,2.0,0.0,2432.0,2.3,0.41072,1.7,0.58928
3,0.0,0.0,3.0,0.0,2443.0,2.3,0.41072,1.7,0.58928
4,0.0,0.0,4.0,0.0,2555.0,2.3,0.41072,1.7,0.58928


### 1. no-bias GLAM

In [11]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM hierarchically...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_nobias_'+sufix+'.npy')):
    glam_nobias.make_model('individual', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_FF2019_nobias'+sufix+'.npy'))
 

Fitting no-bias GLAM hierarchically...
Generating single subject models for 31 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 31 model(s) using NUTS...
  Fitting model 1 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [09:57<00:00,  4.99draws/s]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 31...


...

  Fitting model 30 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:07<00:00, 1505.13draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 31 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:07<00:00, 1543.18draws/s]
The acceptance probability does not match the target. It is 0.5537839386507647, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.6485221449647065, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7142296695566922, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


/!\ Automatically setting parameter precision...


In [12]:
glam_nobias.type

'individual'

In [14]:
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_nobias'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

,SNR,b,gamma,p_error,s,t0,tau,v
0,54.69,1.0,[1.0],0.05,0.002276,[0.0],0.01,0.000053
1,166.29,1.0,[1.0],0.05,0.009831,[0.0],0.02,0.000054
2,26.65,1.0,[1.0],0.05,0.002196,[0.0],0.01,0.000050
3,83.32,1.0,[1.0],0.05,0.009696,[0.0],0.01,0.000038
4,221.73,1.0,[1.0],0.05,0.012437,[0.0],0.01,0.000052
5,133.76,1.0,[1.0],0.05,0.008596,[0.0],0.01,0.000059
6,128.53,1.0,[1.0],0.05,0.008961,[0.0],0.01,0.000065
7,109.23,1.0,[1.0],0.05,0.007703,[0.0],0.49,0.000073
8,121.03,1.0,[1.0],0.05,0.008256,[0.0],0.01,0.000058
9,47.07,1.0,[1.0],0.05,0.003229,[0.0],0.65,0.000097


In [16]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2019_nobias'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_nobias.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_nobias.waic = np.load(str('results/waic/glam_F2019_nobias'+ sufix +'.npy'))

Computing WAIC scores for full model...


/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:211: UserWarning: For one or more samples the posterior variance

In [17]:
# Compute WAICs
np.save(str('results/waic/glam_FF2019_nobias'+ sufix +'.npy'), glam_nobias.waic)

In [18]:
waic_values = pd.DataFrame(glam_nobias.waic)
waic_values.to_csv(str('results/params_estimates/waic_estimate'+sufix+'.csv'))

In [21]:
len(glam_nobias.waic)

31

In [22]:
glam_nobias.waic

array([[2.42758416e+03, 0.00000000e+00, 6.57234681e+02, 1.00000000e+00],
       [1.09866081e+03, 0.00000000e+00, 2.04102169e+00, 1.00000000e+00],
       [4.41720445e+03, 0.00000000e+00, 1.65286155e+03, 1.00000000e+00],
       [1.38841324e+03, 0.00000000e+00, 1.23603561e+02, 1.00000000e+00],
       [1.20699593e+03, 0.00000000e+00, 4.76400810e+01, 1.00000000e+00],
       [1.09719557e+03, 0.00000000e+00, 2.18401590e+00, 1.00000000e+00],
       [1.08769258e+03, 0.00000000e+00, 2.18488773e+00, 1.00000000e+00],
       [1.46161355e+03, 0.00000000e+00, 1.89362685e+02, 1.00000000e+00],
       [1.09285841e+03, 0.00000000e+00, 1.80079397e+00, 1.00000000e+00],
       [1.95933325e+03, 0.00000000e+00, 4.37462022e+02, 1.00000000e+00],
       [1.06800053e+03, 0.00000000e+00, 2.01657393e+00, 1.00000000e+00],
       [1.07435646e+03, 0.00000000e+00, 2.07017726e+00, 1.00000000e+00],
       [3.06155301e+03, 0.00000000e+00, 9.81633879e+02, 1.00000000e+00],
       [1.48500775e+03, 0.00000000e+00, 1.72358442e

In [23]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

np.save(str('results/loo/glam_FF2019_nobias'+ sufix +'.npy'), glam_nobias.loo
)

AttributeError: 'list' object has no attribute 'nchains'

In [24]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_nobias'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
    glam_nobias.prediction.to_csv(str('results/predictions/glam_FF2019_nobias'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_FF2019_nobias'+sufix+'.csv'))

glam_nobias.prediction.head()

Predicting test set data using no-bias GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,subject,trial,repeat,choice,rt,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,0.0,1.0,3428.0,2.3,0.41072,1.7,0.58928
1,0.0,0.0,1.0,0.0,3506.0,2.3,0.41072,1.7,0.58928
2,0.0,0.0,2.0,1.0,3232.0,2.3,0.41072,1.7,0.58928
3,0.0,0.0,3.0,1.0,3975.0,2.3,0.41072,1.7,0.58928
4,0.0,0.0,4.0,1.0,3431.0,2.3,0.41072,1.7,0.58928


## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
glam.plots_pretty.plot_fit(test_data, [glam_full.prediction]);

#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## [END] 